# Pressure interpolation

The pressure interpolation is included in pyremo. Historically, REMO has its own interpolator although theoretically the algorithm is similar to what can be found, e.g., in [metpy](https://unidata.github.io/MetPy/latest/examples/sigma_to_pressure_interpolation.html) or [GeoCAT](https://geocat-comp.readthedocs.io/en/latest/user_api/generated/geocat.comp.interp_hybrid_to_pressure.html#geocat.comp.interp_hybrid_to_pressure). However, REMO's algorithm includes a vertical spline fitting and requires temperature, pressure and orography as input. pyremo provides an API based on xarray and also a command line tool that is similar to the former fortran executable. We show an example of how to use the pyremo API here since it gives more control about input and output formats.

Let's load an example dataset:

In [1]:
import pyremo as pr

ds = pr.tutorial.load_dataset()

The dataset contains a number of 3D variables on model levels (hybrid sigma) that can be interpolated to pressure levels. We will use the temperature here.

In [2]:
from pyremo.prsint import pressure_interpolation

# define pressure levels in hPa
plev = [100, 200, 500, 850, 950]
t_plev = pressure_interpolation(
    ds.T,
    plev=[200, 400, 500, 800, 950],
    t=ds.T,
    ps=ds.PS,
    orog=ds.FIB,
    a=ds.hyai,
    b=ds.hybi,
    keep_attrs=True,
)

   182.09448242187500        294.87936401367188     
           5           5   1.0000000000000000      T       
   182.09448242187500        294.87936401367188     
 fpfl   197.29083813445482        292.79754012361468     


The output dataset `t_plev` has the same structure and meta information as the input dataset. The algorithm utilizes xarray's [apply_ufunc](https://xarray.pydata.org/en/stable/examples/apply_ufunc_vectorize_1d.html) implementation which is particularly useful if the input dataset has a temporal dimension. The pressure interpolation will then also work with dask's vectorization for distributed computing.

In [3]:
t_plev

<xarray.DataArray 'T' (time: 1, plev: 5, rlat: 121, rlon: 129)>
array([[[[213.12659, 213.03859, 212.98166, ..., 218.40222, 218.89198,
          219.50543],
         [213.05632, 213.02272, 212.99855, ..., 218.59741, 219.00497,
          219.39369],
         [212.9752 , 212.97455, 212.97284, ..., 218.6003 , 218.94669,
          219.21333],
         ...,
         [205.92923, 205.80441, 205.6163 , ..., 212.40237, 212.5669 ,
          212.67844],
         [205.76294, 205.64067, 205.45866, ..., 212.3241 , 212.49187,
          212.60616],
         [205.67007, 205.55211, 205.37909, ..., 212.27249, 212.44165,
          212.55777]],

        [[250.49983, 250.1684 , 249.94472, ..., 248.38046, 248.73172,
          249.97137],
         [250.32408, 250.20438, 250.09102, ..., 249.63022, 249.83798,
          250.41566],
         [250.12021, 250.11423, 250.05507, ..., 250.41574, 250.60515,
          250.87677],
...
         [261.37366, 261.39172, 261.4109 , ..., 237.21904, 237.16257,
          237.13774],
         [261.3278 , 261.3635 , 261.39365, ..., 236.9552 , 236.89821,
          236.86525],
         [261.23694, 261.29245, 261.34537, ..., 236.82097, 236.7611 ,
          236.71799]],

        [[290.3318 , 289.80554, 289.6404 , ..., 279.1307 , 276.8874 ,
          276.87778],
         [290.51   , 290.44998, 290.50732, ..., 280.41333, 278.96732,
          278.74188],
         [290.8002 , 290.9886 , 291.1433 , ..., 281.57138, 280.8813 ,
          280.7058 ],
         ...,
         [270.91458, 270.90036, 270.82547, ..., 237.91591, 237.82176,
          237.74191],
         [270.89178, 270.89874, 270.85312, ..., 237.49524, 237.42278,
          237.3607 ],
         [270.8134 , 270.84143, 270.82504, ..., 237.22202, 237.17627,
          237.14983]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2006-01-01
  * rlon     (rlon) float64 -31.73 -31.29 -30.85 -30.41 ... 23.71 24.15 24.59
  * rlat     (rlat) float64 -26.73 -26.29 -25.85 -25.41 ... 25.19 25.63 26.07
  * plev     (plev) float64 2e+04 4e+04 5e+04 8e+04 9.5e+04
Attributes:
    grid_mapping:  rotated_pole
    variable:      T
    description:   temperature
    units:         K
    layer:         110.0
    code:          130

## Command line interface

There is also a command line interface available to run the pressure interpolation on remo output files (NetCDF). If you have installed pyremo using pip or conda, the `prsint` command should be available, e.g.

In [4]:
!prsint -h

usage: prsint [-h] [-v VARIABLES [VARIABLES ...]] [-p PLEV [PLEV ...]] [-s]
              [-id ID] [-cdo CDO_OPTIONS]
              input file [input file ...]

positional arguments:
  input file

optional arguments:
  -h, --help            show this help message and exit
  -v VARIABLES [VARIABLES ...], --variables VARIABLES [VARIABLES ...]
                        list of variables to interpolate (default = ['T',
                        'FI', 'U', 'V', 'QD', 'QW'])
  -p PLEV [PLEV ...], --plev PLEV [PLEV ...]
                        list of pressure levels to interpolate to (default =
                        [100, 200, 500, 850, 950])
  -s, --split           write one pressure level per output file
  -id ID, --id ID       experiment id for output file naming
  -cdo CDO_OPTIONS, --cdo_options CDO_OPTIONS
                        options for using cdo to read input


You can check out the available options. Note, that the remo pressure interpolator used to write one file per pressure level, however, the default of the `prsint` command line interface is to write all pressure levels in one file. You can still write one pressure level per file using the `--split` flag. If you need more control over the in and output formats, we recommend to use the [prsint API](https://pyremo.readthedocs.io/en/stable/generated/pyremo.prsint.pressure_interpolation.html) from python directly.